<a href="https://colab.research.google.com/github/eugenia-eg/freeCodeCamp/blob/main/fcc_cat_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
try:
  # This command only in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [33]:
# Get project files
!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip
!unzip -o cats_and_dogs.zip  # Overwrite without prompt

# Define the path to the dataset
PATH = '/Users/eugeniagalan/Downloads/cats_and_dogs/'

# List contents of the extracted directory to check if the 'test' directory exists
!ls -R cats_and_dogs

# Set directory paths
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

# Check if the test directory exists
if not os.path.exists(test_dir):
    print(f"Test directory does not exist at {test_dir}.")
    print("Creating test directory from train data.")

    # If 'test' doesn't exist, split the 'train' directory into train and test
    # Here, we use 10% of the train data as the test data
    import shutil
    import os

    # Create the test directory and split the data
    os.makedirs(test_dir, exist_ok=True)
    for class_name in os.listdir(train_dir):
        class_train_dir = os.path.join(train_dir, class_name)
        class_test_dir = os.path.join(test_dir, class_name)
        os.makedirs(class_test_dir, exist_ok=True)

        # Get a list of files in the 'train' class directory
        files = os.listdir(class_train_dir)
        test_files = files[:int(len(files) * 0.1)]  # Use the first 10% for testing

        # Move test files to the 'test' directory
        for file in test_files:
            shutil.move(os.path.join(class_train_dir, file), os.path.join(class_test_dir, file))

# Now the test directory should exist, so let's count the files
total_train = sum([len(files) for _, _, files in os.walk(train_dir)])
total_val = sum([len(files) for _, _, files in os.walk(validation_dir)])
total_test = len(os.listdir(test_dir))

# Print file counts for confirmation
print(f"Total training images: {total_train}")
print(f"Total validation images: {total_val}")
print(f"Total test images: {total_test}")

# Variables for pre-processing and training
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150


Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/cats_and_dogs/train/dogs/._dog.531.jpg  
  inflating: cats_and_dogs/train/dogs/dog.519.jpg  
  inflating: __MACOSX/cats_and_dogs/train/dogs/._dog.519.jpg  
  inflating: cats_and_dogs/train/dogs/dog.928.jpg  
  inflating: __MACOSX/cats_and_dogs/train/dogs/._dog.928.jpg  
  inflating: cats_and_dogs/train/dogs/dog.900.jpg  
  inflating: __MACOSX/cats_and_dogs/train/dogs/._dog.900.jpg  
  inflating: cats_and_dogs/train/dogs/dog.914.jpg  
  inflating: __MACOSX/cats_and_dogs/train/dogs/._dog.914.jpg  
  inflating: cats_and_dogs/train/dogs/dog.727.jpg  
  inflating: __MACOSX/cats_and_dogs/train/dogs/._dog.727.jpg  
  inflating: cats_and_dogs/train/dogs/dog.733.jpg  
  inflating: __MACOSX/cats_and_dogs/train/dogs/._dog.733.jpg  
  inflating: cats_and_dogs/train/dogs/dog.754.jpg  
  inflating: __MACOSX/cats_and_dogs/train/dogs/._dog.754.jpg  
  inflating: cats_and_dogs/train/dogs/dog.998.jpg  
  inflating: __MACOSX/cats_an

In [42]:
# Set directory paths
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

# Define batch size and image dimensions
batch_size = 128
IMG_HEIGHT = 150
IMG_WIDTH = 150

# Initialize ImageDataGenerators for training, validation, and testing
train_image_generator = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values between 0 and 1
    rotation_range=45,  # Rotate images up to 45 degrees
    width_shift_range=0.2,  # Shift the image width-wise
    height_shift_range=0.2,  # Shift the image height-wise
    shear_range=0.2,  # Shear the image
    zoom_range=0.2,  # Zoom into the image
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill pixels after transformations
)

validation_image_generator = ImageDataGenerator(rescale=1.0/255)  # Only rescaling for validation
test_image_generator = ImageDataGenerator(rescale=1.0/255)  # Only rescaling for testing

# Create data generators for training, validation, and testing
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'  # Assuming binary classification (cats vs. dogs)
)

val_data_gen = validation_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

test_data_gen = test_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary',
    shuffle=False  # Don't shuffle test data to keep track of images
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/eugeniagalan/Downloads/cats_and_dogs/train'

In [25]:
# 4
def plotImages(images_arr, probabilities = False):
    fig, axes = plt.subplots(len(images_arr), 1, figsize=(5,len(images_arr) * 3))
    if probabilities is False:
      for img, ax in zip( images_arr, axes):
          ax.imshow(img)
          ax.axis('off')
    else:
      for img, probability, ax in zip( images_arr, probabilities, axes):
          ax.imshow(img)
          ax.axis('off')
          if probability > 0.5:
              ax.set_title("%.2f" % (probability*100) + "% dog")
          else:
              ax.set_title("%.2f" % ((1-probability)*100) + "% cat")
    plt.show()

sample_training_images, _ = next(train_data_gen)
plotImages(sample_training_images[:5])


TypeError: 'NoneType' object is not an iterator

In [37]:
# 5
train_image_generator = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values between 0 and 1
    rotation_range=45,  # Rotate images up to 45 degrees
    width_shift_range=0.2,  # Shift the image width-wise
    height_shift_range=0.2,  # Shift the image height-wise
    shear_range=0.2,  # Shear the image
    zoom_range=0.2,  # Zoom into the image
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill pixels after transformations
)



In [27]:
# 6
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)
def plotImages(images_arr):
    fig, axes = plt.subplots(1, len(images_arr), figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

augmented_images = [train_data_gen[0][0][i] for i in range(5)]
plotImages(augmented_images)

NameError: name 'batch_size' is not defined

In [28]:
# 7
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


NameError: name 'IMG_HEIGHT' is not defined

In [29]:
# 8
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=train_data_gen,
    validation_steps=total_val // batch_size
)


NameError: name 'model' is not defined

In [30]:
# 9
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'history' is not defined

In [ ]:
# 11
answers =  [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
            1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
            1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
            1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
            0, 0, 0, 0, 0, 0]

correct = 0

for probability, answer in zip(probabilities, answers):
  if round(probability) == answer:
    correct +=1

percentage_identified = (correct / len(answers)) * 100

passed_challenge = percentage_identified >= 63

print(f"Your model correctly identified {round(percentage_identified, 2)}% of the images of cats and dogs.")

if passed_challenge:
  print("You passed the challenge!")
else:
  print("You haven't passed yet. Your model should identify at least 63% of the images. Keep trying. You will get it!")